In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from vqls import *
from utils import *


In [42]:
func_out = {'sigmoid': sigmoid_t,'tanh': tanh_t,'elu': elu_t, 'relu': relu_t, 'sin':sin_m}
func_list = list(func_out.keys())
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':.0}

pnq = {'3':(3,8),
        '4':(4,16)}
columns=['function', 'nq', 'knot', 'training cost', 'time(s)', 'weights', 'RSS_h', 'RSS_q' ]
tot = {}
for flab, fun in func_out.items():
    for nq, knot in  pnq.values():
        file_path = f"results_{flab}_{nq}.json"
        df = pd.read_json(file_path).drop(['Condition number', 'norm(yk)', 'training_cost', 'exe_time','RSS_h'], axis =1 )
        tmp = []
        for el in df["in_train_weights"]:
             tmp.append(el[-1])
        df["weights"] = tmp
        df = df.drop(["in_train_weights"], axis = 1)
        tot[f"{flab}{nq}"] = df 

In [ ]:
f_i = func_dict[flab]
scaled=False    
label = flab
n_step = pnq[f"{nq}"][1]
x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
y = [fun(value,f_i) for value in xx]

#tck=splrep(x,y,k=1) #coeffs
norm = np.linalg.norm(y)
y = y / norm
matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)


In [58]:
rss=[]
for flab, fun in func_out.items():
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        for i, el in dl.iterrows():
            
            lower = 0.
            upper = 1.

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
            y = [fun(value,f_i) for value in xx]

            #tck=splrep(x,y,k=1) #coeffs
            norm = np.linalg.norm(y)
            y = y / norm
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) #classic

            y_fq=[]
            q = vqls_circuit.solution(el['weights']).real
            y_q=np.dot(matrix,q) #hybrid
                            #Quantum

            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))
            rmse_scal.append(math.sqrt(np.square(np.subtract(y,y_fq)).mean()))
        dl["rmse_scal"] = rmse_scal
        dl["rmse_yc"] = rmse_yc


       RSS_q  seed                                            weights  \
0   3.963544     2  [1.1045460402, 0.2563744746, 2.6760913795, 3.0...   
1   9.433116    27  [2.5609036114, 3.5482101222, 2.8214924623, 4.0...   
2   9.426826    70  [3.1358322811, 2.4786085119, 1.818483076600000...   
3   0.276599    52  [2.5513045036, 0.38891200810000004, -0.1988019...   
4   4.145746    22  [1.9330162929, 2.906664333, 2.5177229883, 1.83...   
5   9.409706    72  [-0.17701141920000002, 2.6813918672, 2.2805960...   
6   3.943581    60  [2.7270898732, 0.8555839589, 2.164968563, 2.13...   
7   4.357991     7  [-0.4807136286, 3.7600193766, 2.1539283566, 2....   
8   3.975778    67  [1.2831687393, 2.8376809003, 2.4117972213, 1.3...   
9   4.098747     1  [3.1298330457, 4.4322975341, 1.0925061653, 0.1...   
10  3.987636    54  [0.7810533275, 3.4293212531, 0.025856068200000...   
11  0.275849    89  [2.8564677766, 0.9024321844000001, -0.23085810...   
12  0.268566    28  [2.7451074065, 2.6239581654, 0.

In [60]:
dk= tot["sigmoid4"]
#dk['RSS_q'] = dk['RSS_q']/dk['knot']
print(dk.pivot( columns = [], values=['rmse_yc','rmse_scal']).sort_values(["rmse_yc"]).to_markdown())

|      |   rmse_yc |   rmse_scal |
|:-----|----------:|------------:|
| (0,) |  0.30803  |   0.0776852 |
| (4,) |  0.308428 |   0.0780663 |
| (7,) |  0.309383 |   0.0781614 |
| (2,) |  0.660615 |   0.345134  |
| (1,) |  0.671942 |   0.353697  |
| (3,) |  0.936407 |   0.565645  |
| (5,) |  0.93789  |   0.567159  |
| (6,) |  0.938195 |   0.567497  |


In [8]:
flab = 'relu'
fun = relu_t
lower = 0. 
upper = 1. 
f_i = func_dict[flab]
scaled=False    
label = flab
n_step = el.knot
x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
y = [fun(value,f_i) for value in x]

tck=splrep(x,y,k=1) #coeffs
norm = np.linalg.norm(y)
y = y / norm
ax[i].plot(np.linspace(0,1,len(y)),y, label=f"Original function scaled {n_step}", color = '#5c4d7d',linewidth=2.5)

ax[i].axhline(0, color="#222222")
matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

vqls_circuit = VQLS(matrix,v_norm,el['nq'],opt='COBYLA') 

c = np.linalg.solve(matrix,vector)

y_c=np.dot(matrix,c) #classic
ax[i].plot(np.linspace(0,1,len(y_c)),y_c, label="classical spline", color = '#1780a1',linewidth=2.5)

y_fq=[]
q = vqls_circuit.solution(el['weights']).real
y_q=np.dot(matrix,q) #hybrid
                #Quantum

for a in matrix:
    y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
rss.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))


In [9]:
y_fq

[tensor(-0.24159205, requires_grad=True),
 tensor(-0.27983433, requires_grad=True),
 tensor(-0.4385123, requires_grad=True),
 tensor(-0.47483765, requires_grad=True),
 tensor(-0.51361569, requires_grad=True),
 tensor(-0.49873931, requires_grad=True),
 tensor(-0.40872349, requires_grad=True),
 tensor(-0.30831969, requires_grad=True),
 tensor(-0.15331156, requires_grad=True),
 tensor(-0.07351244, requires_grad=True),
 tensor(-0.01786041, requires_grad=True),
 tensor(-0.0303805, requires_grad=True),
 tensor(-0.03717123, requires_grad=True),
 tensor(-0.15253645, requires_grad=True),
 tensor(-0.16465614, requires_grad=True),
 tensor(-0.15439434, requires_grad=True)]